In [11]:
import numpy as np
import pandas as pd
from scipy.stats import gmean
from tqdm import tqdm

from label_to_cat import LABEL_TO_CAT

BATCHES_COUNT = 12091
TEST_SIZE = 1768182

In [12]:
res50_path = '../input/resnet50_test_corr.h5'
res101_path = '../input/resnet101_test.h5'
dense_path = '../input/dense_test_corr.h5'
d1_path = '../input/dense_test_epoch19.h5'
se_inc_path = '../input/se_inc_test_TTA.h5'
inc_path = '../input/inc3_test_TTA.h5'

In [3]:
%%time
store_res50 = pd.HDFStore(res50_path)
print('{} tables in storage'.format(len(store_res50)))

12091 tables in storage
CPU times: user 41.4 s, sys: 308 ms, total: 41.7 s
Wall time: 41.7 s


In [4]:
%%time
store_res101 = pd.HDFStore(res101_path)
print('{} tables in storage'.format(len(store_res101)))

12091 tables in storage
CPU times: user 42.7 s, sys: 308 ms, total: 43 s
Wall time: 43 s


In [5]:
%%time
store_dense = pd.HDFStore(dense_path)
print('{} tables in storage'.format(len(store_res101)))

12091 tables in storage
CPU times: user 42.3 s, sys: 252 ms, total: 42.5 s
Wall time: 42.5 s


In [6]:
%%time
store_d1 = pd.HDFStore(d1_path)
print('{} tables in storage'.format(len(store_d1)))

12091 tables in storage
CPU times: user 41.9 s, sys: 224 ms, total: 42.1 s
Wall time: 42.1 s


In [7]:
%%time
store_se_inc = pd.HDFStore(se_inc_path)
print('{} tables in storage'.format(len(store_se_inc)))

12091 tables in storage
CPU times: user 42.6 s, sys: 384 ms, total: 43 s
Wall time: 43 s


In [8]:
%%time
store_inc = pd.HDFStore(inc_path)
print('{} tables in storage'.format(len(store_inc)))

24181 tables in storage
CPU times: user 1min 27s, sys: 780 ms, total: 1min 27s
Wall time: 1min 27s


In [9]:
stores = [store_res50, store_res101, store_dense, store_d1, store_se_inc, store_inc]
# keys = store_res50.keys()
count = len(stores[0])
print("Count:", count)

Count: 12091


In [7]:
# stores = [store_res101]

In [10]:
preds = []
cur_id = None
one_product_preds = None

def select_tables_SIX(stores, index):
    tables = []
    key = 'd' + str(index)
    for i in range(len(stores) - 1):
        tables.append(stores[i].select(key))
    k1 = 'd' + str(2*index)
    k2 = 'd' + str(2*index + 1)
    if index != 12090:
        tables.append(
            pd.concat([stores[len(stores) - 1].select(k1), stores[len(stores) - 1].select(k2)],
                     ignore_index=True)
        )
    else:
        tables.append(stores[len(stores) - 1].select(k1))
    return tables
        
def get_product_id(tables, index):
    assert len(tables) > 0
    product_id = tables[0].pr_id.iloc[index]
    for i in range(1, len(tables)):
        alt = tables[i].pr_id.iloc[index]
        assert alt == product_id
    return product_id

def get_new_sum_gmean(tables, ind):
    sums = [table[ind : ind +1].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
                           for table in tables]
    return gmean(sums)

def get_cat_id(one_pr_preds):
    #averaged = one_pr_preds[0]
    #for i in range(1, len(one_pr_preds)):
    #    averaged += one_product_preds[i] * (1. - 0.3*i)
    #id_ = int(averaged.argmax()) - 1
    #assert 0 <= id_ < 5270
    #return id_
    return gmean(one_pr_preds).argmax()

for index in tqdm(range(count)): 
    tables = select_tables_SIX(stores, index)
    for i in range(tables[0].shape[0]):
        product_id = get_product_id(tables, i)
        new_sum = get_new_sum_gmean(tables, i)
        if cur_id == product_id:
            assert one_product_preds is not None
            one_product_preds.append(new_sum)
            continue
        elif cur_id is not None:
            assert one_product_preds is not None
            cat = LABEL_TO_CAT[get_cat_id(one_product_preds)]
            preds.append((int(cur_id), cat))
            one_product_preds = None
        
        cur_id = product_id
        one_product_preds = [new_sum]
        
if cur_id is not None:
    cat = LABEL_TO_CAT[get_cat_id(one_product_preds)]
    preds.append((int(cur_id), cat))
            
preds_df = pd.DataFrame(data=preds, columns=['_id','category_id'])
print('Dataframe shape {}; products in test {}'.format(preds_df.shape, TEST_SIZE))
print('Duplicates {}; affect accuracy {}'.format(preds_df.shape[0] - TEST_SIZE,
                                                float(preds_df.shape[0] - TEST_SIZE) / TEST_SIZE))
no_dupls = preds_df.drop_duplicates(subset='_id', inplace=False)
print('No duplicated shape: {}; expected rows count {}'.format(no_dupls.shape, TEST_SIZE))

no_dupls.to_csv('../submit/SIX_MODEL_GMEAN.csv', index=False)

  0%|          | 0/12091 [00:00<?, ?it/s]/home/ikibardin/anaconda3/envs/python3.5/lib/python3.5/site-packages/scipy/stats/stats.py:306: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
100%|██████████| 12091/12091 [11:00:37<00:00,  3.28s/it]  


Dataframe shape (1768182, 2); products in test 1768182
Duplicates 0; affect accuracy 0.0
No duplicated shape: (1768182, 2); expected rows count 1768182
